In [7]:
from abjad import *

# Bottom-up Work with Leaves (Notes, Chords, Rests, and Skips)

## Intro 

Abjad extends the Python programming language with an object-oriented model of common practice music notation. To work with this model, we'll need to understand two key concepts: 1) object-orientation and 2) the leaf, container, spanner, idnicator tree model of a score.

## An Object-oriented Model of Music Notation 

What does it mean that we have an object model of music notation? To answer this question, we'll first make a single note by calling the constructor function of the Note class.

In [9]:
grape = Note()

We now have a new note called "grape." If we forget what we've made because of our whimsical name, we can type the name of our note into the interpreter to see what it is.

In [10]:
grape

Note("c'4")

Sure enough, it's a note, with a LilyPond string of middle C and a quarter-note duration, which are unspecified default values.

So what happened when we made our note? In our object model, we have a class called Note that models a musical note. A class is a template for a certain kind of thing. The Note class models the characteristics of a note (attributes). To make a new note (a new instance of the Note class), we called our class's factory function, Note(), and assigned the returned instance to a variable, named whatever we like (on the left side of the equals sign). You will see this pattern of instantiating objects through right-to-left assignment over and over, throughout the system.

Once we have our note instance, we can access characteristics about our note that we'd expect it to have using dot chaining syntax, where we start with our variable name and continue, after a dot, with the attribute we'd like to get:

In [11]:
grape.written_pitch

NamedPitch("c'")

In [13]:
grape.written_duration

Duration(1, 4)

In addition to getting these attributes, we can also write them using the same syntax -- here we change the pitch of our note to be inaudibly high:

In [14]:
grape.written_pitch = "fs'''''''''''''''"

And now, when we ask the interpreter what grape is, its written pitch has been changed.

In [15]:
grape

Note("fs'''''''''''''''4")

We can do the same with our note's duration, and see the change in the note's LilyPond string.

In [16]:
grape.written_duration = (1,2)

In [17]:
grape

Note("fs'''''''''''''''2")

The many classes in Abjad give you models of all common notational symbols. You can see a list of attributes and methdos for each class at http://abjad.mbrsi.org/api/ .

For example, to find the Note class, we look in the scoretools package in the API reference. scoretools contains the most commonly used score elements. Under "leaves" we find the Note class. When we click Note, we're taken to the reference page for Note. The attributes summary shows us which characteristics of a note have been modelled. The read/write properties portion, often with examples, shows us which attributes we can read/write.

## Other Leaf Classes

We notice in the API reference that there are other classes listed under scoretools-Leaves. We can create Rests, Chords, Skips in the same way we made our note.

In [18]:
rest = Rest()

In [19]:
rest

Rest('r4')

We get a default value of a quarter-note, but we can also pass in an argument -- either a duration or a LilyPond string -- to choose something other than the default.

In [20]:
rest = Rest("r8..")

In [21]:
rest

Rest('r8...')

In [24]:
rest = Rest((7,32))

In [25]:
rest

Rest('r8..')

Likewise, we can make notes other than a middle C quarter note by passing in arguments. We can make a new note using a LilyPond string,

In [26]:
note = Note("ds'32")

In [27]:
note

Note("ds'32")

Or by giving two comma-separated pitch and duration arguments:

In [30]:
note = Note(3, (1,32))

Note that Abjad represents pitch numerically by equating 0 to middle C. A pitch of 3 corresponds to three half-steps above middle C.

In [31]:
note

Note("ef'32")

Hm - we got a flat, but we wanted a sharp. Good thing we can change Abjad's default accidental spellings:

In [32]:
abjad_configuration.set_default_accidental_spelling('sharps')

In [34]:
note = Note(3, (1,32))

In [35]:
note

Note("ds'32")

In [ ]:
Chords work similarly:

In [36]:
chord = Chord("<c' e' g'>8.")

In [37]:
chord

Chord("<c' e' g'>8.")

In [38]:
chord = Chord([0,4,7], (3,16))

In [39]:
chord

Chord("<c' e' g'>8.")

As do skips (blank spaces that occupy a specified duration -- like rests without the drawn symbol.)

In [40]:
skip = Skip("s8.")

In [41]:
skip

Skip('s8.')

In [42]:
skip = Skip((3,16))

In [43]:
skip

Skip('s8.')

And remember, we can get back to the attributes of our leaves using dot chaining syntax.

In [44]:
skip.written_duration

Duration(3, 16)

In [45]:
chord.written_duration

Duration(3, 16)

In [46]:
rest.written_duration

Duration(7, 32)

## The Leaf, Container Spanner,  Indicator Model

Be sure you understand the explanation of Abjad's score tree model, as illustated here: http://abjad.mbrsi.org/core_concepts/lcsi.html
        
In summary --
Leaves: notes, chords, rests, and skips, as above. These can be arranged together sequentially or simultaneously in

Containers: Tuplets, Voices, Staffs, and Scores. Containers can contain leaves directly (a Staff containing Notes) or can contain other containers hierarchically (a Score containing a StaffGroup containing two Staffs which both contain Leaves):





In [91]:
staff1 = Staff("c' d' e' f'")

In [92]:
staff2 = Staff("f' e' d' c'")

In [93]:
group = StaffGroup([staff1, staff2])

In [94]:
score = Score([group])

In [95]:
show(score)

Spanners: anything that spans from one component to another, i.e. a phrasing slur that starts at one note and ends at another note, spanning everything in between. Here, we slur all the notes in our staff1 container by attaching an instance of the PhrasingSlur class to all the notes contained in staff1:

In [88]:
slur = spannertools.PhrasingSlur()

In [89]:
attach(slur, staff1[:])

In [90]:
show(score)

Indicators: anything that attaches to a single component. Some indicators become effective at that point and remain so until a change (dynamic markings, time signatures, tempo markings) and some impact only the note to which they attach (articulations). Here we use indexing to attach an articulation to the second component in our staff2 container (remember that indexes count from 0, so index 1 is the second component in the container).

In [99]:
staccato = Articulation('staccato')

In [100]:
attach(staccato, staff2[1])

In [101]:
show(score)

It might be the case that we want to apply the same indicator to a consecutive series of leaves. Note that we can use iteration to do the same thing to multiple components sequentially -- here we use slice notation to apply a staccato marking to every leaf in staff1 after the first note:

In [102]:
for leaf in staff1 [1:]:
    staccato = Articulation('staccato')
    attach(staccato, leaf)

In [103]:
show(score)

## Making Many Leaves

We can use anything in Python -- from built-in libraries to any of its thousands of external libraries -- to create leaves. Python's list comprehension syntax allows us to describe lists more easily:

In [108]:
notes = [Note(x,(1,8)) for x in range(24+1)]

The above list comprehension means, "a list of eighth notes with pitch values from 1 through 25."

In [109]:
staff = Staff(notes)

In [110]:
show(staff)

We can use external libraries, too -- here we generate a hundred random numbers and use them as pitch values:

In [118]:
import random

In [123]:
numbers = [random.randrange(0,25) for x in range(100)]

In [124]:
numbers

[11,
 9,
 7,
 4,
 2,
 0,
 21,
 5,
 7,
 6,
 5,
 9,
 7,
 17,
 3,
 7,
 11,
 19,
 3,
 14,
 5,
 6,
 22,
 18,
 7,
 13,
 22,
 16,
 20,
 21,
 11,
 10,
 20,
 3,
 1,
 15,
 21,
 24,
 14,
 13,
 21,
 18,
 23,
 14,
 24,
 17,
 21,
 6,
 4,
 21,
 13,
 23,
 17,
 14,
 18,
 16,
 8,
 4,
 17,
 14,
 16,
 10,
 7,
 13,
 24,
 19,
 21,
 14,
 10,
 14,
 10,
 23,
 2,
 7,
 14,
 23,
 1,
 5,
 22,
 23,
 0,
 6,
 22,
 1,
 21,
 11,
 2,
 22,
 2,
 23,
 22,
 17,
 0,
 7,
 20,
 6,
 20,
 4,
 3,
 18]

And now we can turn them into notes with a list comprehension:

In [125]:
notes = [Note(x,(1,8)) for x in numbers]

In [126]:
staff = Staff(notes)

In [127]:
show(staff)

N.B. we've just been using integers as chromatic pitch numbers, but we could use them as anything else we like (rhythmic durations, for example). Quarter-tone accidentals are also possible, using .5 resolution (anything other than 0 or .5 will be correspondingly rounded up):

In [132]:
numbers = [x / 2.0 for x in numbers]

In [133]:
numbers

[2.75,
 2.25,
 1.75,
 1.0,
 0.5,
 0.0,
 5.25,
 1.25,
 1.75,
 1.5,
 1.25,
 2.25,
 1.75,
 4.25,
 0.75,
 1.75,
 2.75,
 4.75,
 0.75,
 3.5,
 1.25,
 1.5,
 5.5,
 4.5,
 1.75,
 3.25,
 5.5,
 4.0,
 5.0,
 5.25,
 2.75,
 2.5,
 5.0,
 0.75,
 0.25,
 3.75,
 5.25,
 6.0,
 3.5,
 3.25,
 5.25,
 4.5,
 5.75,
 3.5,
 6.0,
 4.25,
 5.25,
 1.5,
 1.0,
 5.25,
 3.25,
 5.75,
 4.25,
 3.5,
 4.5,
 4.0,
 2.0,
 1.0,
 4.25,
 3.5,
 4.0,
 2.5,
 1.75,
 3.25,
 6.0,
 4.75,
 5.25,
 3.5,
 2.5,
 3.5,
 2.5,
 5.75,
 0.5,
 1.75,
 3.5,
 5.75,
 0.25,
 1.25,
 5.5,
 5.75,
 0.0,
 1.5,
 5.5,
 0.25,
 5.25,
 2.75,
 0.5,
 5.5,
 0.5,
 5.75,
 5.5,
 4.25,
 0.0,
 1.75,
 5.0,
 1.5,
 5.0,
 1.0,
 0.75,
 4.5]

In [134]:
notes = [Note(x,(1,8)) for x in numbers]

In [135]:
staff = Staff(notes)

In [136]:
show(staff)

## Conclusion

This tutorial outlines the basics of bottom-up work with Abjad. First, you create leaves by some mechanism, and then you put these leaves in containers, add spanners and indicators to them, etc., rather manually. 